In [2]:
import os
# !pip install pyarrow==15.0.2
# !pip install optuna
# !pip install optuna_integration
# !pip install evaluate

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ['TORCH_CUDA_ALLOC_SYNC'] = '1'

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizerFast, TrainerCallback
from torch.utils.data import Dataset
from torch import tensor, cuda
from torch.nn.functional import softmax
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import evaluate
import optuna
from optuna.pruners import MedianPruner
from optuna.integration import TensorBoardCallback
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00


In [3]:
class CSVDataset(Dataset):
    def __init__(self, bindings, labels=None, device='cpu'):
        self.bindings = bindings
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.bindings["input_ids"])

    def __getitem__(self, idx):
        item = {k: tensor(v[idx]).to(self.device) for k, v in self.bindings.items()}
        if self.labels:
            item['labels'] = tensor(self.labels[idx] - 1).to(self.device)
        return item

class OptunaPruningCallback(TrainerCallback):
    def __init__(self, trial, metric_name):
        self.trial = trial
        self.metric_name = metric_name

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        epoch = math.ceil(state.epoch)
        value = metrics.get(self.metric_name)
        self.trial.report(value, epoch)
        if self.trial.should_prune():
            raise optuna.TrialPruned()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
model_name = "onlplab/alephbert-base"
num_labels = 20
output_dir = "test_trainer"
eval_strategy = "epoch"
csv_file = "updated_final.csv"
device = 'cuda:0' if cuda.is_available() else 'cpu'
metric = evaluate.load("accuracy")
tokenizer = BertTokenizerFast.from_pretrained(model_name)

ds = pd.read_csv(csv_file, sep='\t')
ds = ds.dropna()

X = list("[CLS]" + ds['sentence'] + "[SEP]" + ds['part'] + "[SEP]")
y = list(ds['category'].astype(int))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

train_ds = CSVDataset(X_train_tokenized, y_train, device=device)
test_ds = CSVDataset(X_test_tokenized, y_test, device=device)

print(f'{ds.shape[0]} entries in dataset')
ds.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/545k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

987 entries in dataset


,sentence,part,category
0,"החקלאי יצא לרסס את מטע הזיתים שלו , ומשלא שב ל...",מטע הזיתים,12.0
1,"החקלאי יצא לרסס את מטע הזיתים שלו , ומשלא שב ל...",קריאות הטלפון,11.0
2,"החקלאי יצא לרסס את מטע הזיתים שלו , ומשלא שב ל...",רוח חיים,18.0
3,"גופתו של חקלאי בן חמישים ושלוש , תושב כפר סבא ...",בן חמישים ושלוש,17.0
4,"גופתו של חקלאי בן חמישים ושלוש , תושב כפר סבא ...",תושב כפר סבא,16.0


In [6]:
def optimize_hyperparameters():
    print(f"Optimizing hyperparameters for {model_name} on {device} with {csv_file} dataset")

    def objective(trial):
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=20)
        learning_rate=trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
        weight_decay=trial.suggest_float("weight_decay", 1e-4, 1e-1)
        per_device_train_batch_size=trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32, 64, 128])
        warmup_steps=trial.suggest_int("warmup_steps", 0, 500)
        warmup_ratio=trial.suggest_float("warmup_ratio", 0.0, 0.2)
        num_train_epochs=trial.suggest_int("num_train_epochs", 2, 10)

        training_args = TrainingArguments(
            output_dir="./results",
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            per_device_train_batch_size=per_device_train_batch_size,
            warmup_steps=warmup_steps,
            warmup_ratio=warmup_ratio,
            num_train_epochs=num_train_epochs,
            eval_strategy="epoch",
            logging_strategy="epoch",
            save_strategy="no",
            save_total_limit=2,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=test_ds,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[OptunaPruningCallback(trial, "eval_accuracy")],
        )

        trainer.train()
        out = trainer.evaluate()
        return out['eval_accuracy']

    study = optuna.create_study(direction="maximize", pruner=MedianPruner())
    study.optimize(objective, n_trials=300)
    print("Best Hyperparameters:\n", study.best_params)
    return study

study = optimize_hyperparameters()

[I 2024-08-28 00:23:16,637] A new study created in memory with name: no-name-11db6f31-c266-4a40-b8d8-ab2604577ead


Optimizing hyperparameters for onlplab/alephbert-base on cuda:0 with updated_final.csv dataset


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,3.016300,2.909772,0.116162
2,2.760200,2.557595,0.227273
3,2.445100,2.326430,0.202020
4,2.277700,2.249166,0.217172
5,2.156800,2.184335,0.222222
6,1.979200,2.131437,0.282828
7,1.716500,2.116848,0.373737


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 7 is already reported.
  warnings.warn(
[I 2024-08-28 00:24:47,336] Trial 0 finished with value: 0.37373737373737376 and parameters: {'learning_rate': 1.1538464302436588e-05, 'weight_decay': 0.034981821374836304, 'per_device_train_batch_size': 16, 'warmup_steps': 358, 'warmup_ratio': 0.08213525023749946, 'num_train_epochs': 7}. Best is trial 0 with value: 0.37373737373737376.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,2.452600,2.243027,0.262626


[I 2024-08-28 03:12:14,208] Trial 298 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,2.440200,2.247483,0.262626


[I 2024-08-28 03:12:46,600] Trial 299 pruned. 


Best Hyperparameters:
 {'learning_rate': 4.867915819336374e-05, 'weight_decay': 0.056421880826897274, 'per_device_train_batch_size': 8, 'warmup_steps': 92, 'warmup_ratio': 0.04691041160072529, 'num_train_epochs': 3}


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=20)
best_params = {'learning_rate': 4.867915819336374e-05, 'weight_decay': 0.056421880826897274, 'per_device_train_batch_size': 8, 'warmup_steps': 92, 'warmup_ratio': 0.04691041160072529, 'num_train_epochs': 3}
training_args = TrainingArguments(
    output_dir="output",
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    warmup_steps=best_params['warmup_steps'],
    warmup_ratio=best_params['warmup_ratio'],
    num_train_epochs=best_params['num_train_epochs'],
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
def load_model(save=False):
    if save:
        trainer.save_model('./trained_model')
    return AutoModelForSequenceClassification.from_pretrained('./trained_model')

trained_model = load_model(save=True)
trained_model.evaluate()


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.4165635108947754, 'eval_accuracy': 0.3465346534653465, 'eval_runtime': 4.9265, 'eval_samples_per_second': 20.502, 'eval_steps_per_second': 2.639, 'epoch': 4.0}


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
sentence = "קובי נסע לכיכר הרצל בלוד"
# sentence = "זרקו שם מטען חבלה"
input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
output = trained_model(**input)
print(output)
pred = softmax(output.logits, dim=-1).detach().numpy()
print(pred)
print(np.argmax(pred) + 1)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1365, -0.1568, -0.4376,  0.1788, -0.5550, -0.3552, -0.6783,  0.0173,
         -1.0358,  0.6358, -0.6998, -1.8162, -0.4686,  2.8654,  4.7651,  0.1125,
         -0.6668, -1.7059,  0.1414, -0.9894]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[[0.00772378 0.00575987 0.00435009 0.00805722 0.00386803 0.0047237
  0.00341949 0.00685559 0.00239168 0.01272504 0.00334669 0.00109589
  0.00421709 0.11828709 0.79068893 0.00754044 0.00345895 0.00122368
  0.00776161 0.00250517]]
15
